In [1]:
from keras.layers import Input, Dense, Flatten, Dropout
from keras.models import Model
from keras.layers.convolutional import Conv2D
from data.data import load_data
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [2]:
data1 = pd.read_csv('./data/recnn_no_augmentation.csv')

x_vals = []
merged = np.stack((data1['prediction'], data1['prediction'], data1['prediction'], data1['prediction'], data1['prediction'], data1['prediction'], data1['prediction'], data1['prediction'], data1['prediction'], data1['prediction'], data1['prediction'], data1['prediction'], data1['prediction'], data1['prediction'], data1['prediction'], data1['prediction']), axis=1)
for elem in merged:
    inverted = np.stack((elem, 1-elem), axis = 1)
    inverted = np.expand_dims(inverted, axis=2)
    x_vals.append(inverted)
x_vals = np.array(x_vals)
    
(x_train, y_train, meta_train), (x_valid, y_valid, meta_valid), (x_test, y_test, meta_test) = load_data()
y_vals = y_valid[0:10]

print(np.array(x_vals).shape)

(100, 16, 2, 1)


In [86]:
#inputs = Input(shape=(number of models,number of classes,1))
def build_ensemble_model(model_amount):
    inputs = Input(shape=(model_amount,2,1))
    x = Conv2D(filters=8, kernel_size=(3, 1))(inputs)
    x = Conv2D(16, (3, 1))(x)
    x = Conv2D(32, (3, 1))(x)
    x = Conv2D(64, (3, 1))(x)
    x = Flatten()(x)
    x = Dense(1024)(x)
    x = Dense(2, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=x)
    print(model.summary())
    return model

In [87]:
model = build_ensemble_model(16)
model.compile('adam', 'binary_crossentropy')
model.fit(x=x_vals, y=y_vals, batch_size=2, epochs=1, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 16, 2, 1)          0         
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 14, 2, 8)          32        
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 12, 2, 16)         400       
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 10, 2, 32)         1568      
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 8, 2, 64)          6208      
_________________________________________________________________
flatten_10 (Flatten)         (None, 1024)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 1024)              1049600   
__________

ValueError: Error when checking target: expected dense_20 to have 2 dimensions, but got array with shape (10, 1, 1, 1)

In [ ]:
data1.to_csv('./data/submission_ensemble_2.csv', index=False)